# Fantasy Football Draft Decision Dashboard

A focused, decision-support dashboard that shows only what matters for your next draft pick. Replaces overwhelming matrix visualization with compact, actionable insights.

In [10]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from IPython.core.display import display_html
import warnings
warnings.filterwarnings('ignore')

## 1. Core Calculation Engine (Preserved from Original)

In [11]:
def load_espn_data():
    """Load ESPN projections CSV data"""
    df = pd.read_csv('data/espn_projections_20250814.csv')
    return df

def calculate_availability(player_rank, draft_position, std_dev=3):
    """Calculate probability that a player is available at a given draft position"""
    if draft_position <= 0:
        return 1.0
    # Probability = 1 - CDF(draft_position, mean=player_rank, std=std_dev)
    prob = 1 - stats.norm.cdf(draft_position, loc=player_rank, scale=std_dev)
    return max(0, min(1, prob))

# Load the data
df = load_espn_data()
print(f"Loaded {len(df)} players")
print(df.head())

Loaded 300 players
   overall_rank position position_rank       player_name team  salary_value  \
0             1       WR           WR1     Ja'Marr Chase  CIN            57   
1             2       RB           RB1    Bijan Robinson  ATL            56   
2             3       WR           WR2  Justin Jefferson  MIN            55   
3             4       RB           RB2    Saquon Barkley  PHI            55   
4             5       RB           RB3      Jahmyr Gibbs  DET            54   

   bye_week  
0        10  
1         5  
2         6  
3         9  
4         8  


## 2. Enhanced Data Processing for Dashboard

In [12]:
def calculate_player_metrics(df, my_picks=None, drafted_players=None):
    """Calculate enhanced metrics for each player including availability at my picks"""
    if my_picks is None:
        my_picks = []
    if drafted_players is None:
        drafted_players = set()
    
    enhanced_data = []
    
    for _, player in df.iterrows():
        if player['player_name'] in drafted_players:
            continue  # Skip drafted players
            
        player_rank = player['overall_rank']
        
        # Calculate probability at next available pick
        next_pick_prob = 0.0
        if my_picks:
            next_pick = min([p for p in my_picks if p > 0])  # Next upcoming pick
            next_pick_prob = calculate_availability(player_rank, next_pick)
        
        # Calculate median pick (50% probability point)
        median_pick = player_rank
        
        # Calculate 10th and 90th percentile picks for range
        p10_pick = max(1, stats.norm.ppf(0.9, loc=player_rank, scale=3))
        p90_pick = stats.norm.ppf(0.1, loc=player_rank, scale=3)
        
        enhanced_data.append({
            'player_name': player['player_name'],
            'position': player['position'],
            'position_rank': player['position_rank'],
            'overall_rank': player['overall_rank'],
            'team': player['team'],
            'salary_value': player['salary_value'],
            'bye_week': player['bye_week'],
            'next_pick_prob': next_pick_prob,
            'median_pick': median_pick,
            'p10_pick': int(p10_pick),
            'p90_pick': int(max(1, p90_pick)),
            'pick_range': f"{int(max(1, p90_pick))}-{int(p10_pick)}"
        })
    
    return pd.DataFrame(enhanced_data)

# Example: Calculate metrics with sample "my picks"
my_draft_picks = [8, 17, 32, 41, 56, 65, 80, 89]  # 8-team league, pick 8
enhanced_df = calculate_player_metrics(df, my_picks=my_draft_picks)
print(f"Enhanced data for {len(enhanced_df)} available players")
print(enhanced_df.head())

Enhanced data for 300 available players
        player_name position position_rank  overall_rank team  salary_value  \
0     Ja'Marr Chase       WR           WR1             1  CIN            57   
1    Bijan Robinson       RB           RB1             2  ATL            56   
2  Justin Jefferson       WR           WR2             3  MIN            55   
3    Saquon Barkley       RB           RB2             4  PHI            55   
4      Jahmyr Gibbs       RB           RB3             5  DET            54   

   bye_week  next_pick_prob  median_pick  p10_pick  p90_pick pick_range  
0        10        0.009815            1         4         1        1-4  
1         5        0.022750            2         5         1        1-5  
2         6        0.047790            3         6         1        1-6  
3         9        0.091211            4         7         1        1-7  
4         8        0.158655            5         8         1        1-8  


## 3. Compact Summary Table (Primary View)

In [13]:
def create_summary_table(enhanced_df, position_filter=None, limit=20):
    """Create compact, sortable summary table"""
    
    # Filter by position if specified
    if position_filter and position_filter != 'ALL':
        filtered_df = enhanced_df[enhanced_df['position'] == position_filter].copy()
    else:
        filtered_df = enhanced_df.copy()
    
    # Sort by overall rank (best players first)
    filtered_df = filtered_df.sort_values('overall_rank').head(limit)
    
    # Create display DataFrame
    display_df = filtered_df[['player_name', 'position', 'overall_rank', 'next_pick_prob', 'median_pick', 'pick_range']].copy()
    
    # Format columns
    display_df['Player'] = display_df['player_name'] + ' (' + display_df['position'] + ')'
    display_df['ESPN Rank'] = display_df['overall_rank']
    display_df['P(at my pick)'] = (display_df['next_pick_prob'] * 100).round(1).astype(str) + '%'
    display_df['Median Pick'] = display_df['median_pick'].round(0).astype(int)
    display_df['Range (10-90%)'] = display_df['pick_range']
    
    # Select final columns
    final_df = display_df[['Player', 'ESPN Rank', 'P(at my pick)', 'Median Pick', 'Range (10-90%)']].copy()
    
    # Apply color coding based on probability
    def color_probability(val):
        if '%' in str(val):
            prob = float(val.replace('%', ''))
            if prob >= 70:
                return 'background-color: #d4edda; color: #155724'  # Green
            elif prob >= 30:
                return 'background-color: #fff3cd; color: #856404'  # Yellow
            else:
                return 'background-color: #f8d7da; color: #721c24'  # Red
        return ''
    
    styled_df = final_df.style.applymap(color_probability, subset=['P(at my pick)'])
    
    return styled_df

# Create and display summary table
summary_table = create_summary_table(enhanced_df, limit=15)
print("TOP AVAILABLE PLAYERS - Decision Support Summary")
print("=" * 60)
display(summary_table)

TOP AVAILABLE PLAYERS - Decision Support Summary


,Player,ESPN Rank,P(at my pick),Median Pick,Range (10-90%)
0,Ja'Marr Chase (WR),1,1.0%,1,1-4
1,Bijan Robinson (RB),2,2.3%,2,1-5
2,Justin Jefferson (WR),3,4.8%,3,1-6
3,Saquon Barkley (RB),4,9.1%,4,1-7
4,Jahmyr Gibbs (RB),5,15.9%,5,1-8
5,CeeDee Lamb (WR),6,25.2%,6,2-9
6,Christian McCaffrey (RB),7,36.9%,7,3-10
7,Puka Nacua (WR),8,50.0%,8,4-11
8,Malik Nabers (WR),9,63.1%,9,5-12
9,Amon-Ra St. Brown (WR),10,74.8%,10,6-13


## 4. Player Cards Layout (Mobile-Friendly)

In [14]:
def create_player_cards(enhanced_df, position_filter=None, limit=12):
    """Create mobile-friendly player cards with key metrics"""
    
    # Filter and sort data
    if position_filter and position_filter != 'ALL':
        filtered_df = enhanced_df[enhanced_df['position'] == position_filter].copy()
    else:
        filtered_df = enhanced_df.copy()
    
    filtered_df = filtered_df.sort_values('overall_rank').head(limit)
    
    cards_html = []
    
    for _, player in filtered_df.iterrows():
        prob = player['next_pick_prob'] * 100
        
        # Color coding based on probability
        if prob >= 70:
            card_color = "#d4edda"
            border_color = "#c3e6cb" 
            text_color = "#155724"
        elif prob >= 30:
            card_color = "#fff3cd"
            border_color = "#ffeeba"
            text_color = "#856404"
        else:
            card_color = "#f8d7da"
            border_color = "#f5c6cb"
            text_color = "#721c24"
        
        card_html = f"""
        <div style="
            background-color: {card_color};
            border: 2px solid {border_color};
            border-radius: 8px;
            padding: 12px;
            margin: 8px;
            min-width: 200px;
            max-width: 300px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        ">
            <div style="font-weight: bold; font-size: 16px; color: {text_color}; margin-bottom: 6px;">
                {player['player_name']}
            </div>
            <div style="color: #666; font-size: 12px; margin-bottom: 8px;">
                {player['position']}{player['position_rank']} • {player['team']} • Bye: {player['bye_week']}
            </div>
            <div style="font-size: 14px; margin-bottom: 4px;">
                <strong>ESPN Rank:</strong> #{player['overall_rank']}
            </div>
            <div style="font-size: 14px; margin-bottom: 4px;">
                <strong>P(at my pick):</strong> <span style="color: {text_color}; font-weight: bold;">{prob:.1f}%</span>
            </div>
            <div style="font-size: 14px; margin-bottom: 4px;">
                <strong>Expected Pick:</strong> {player['median_pick']:.0f}
            </div>
            <div style="font-size: 12px; color: #666;">
                <strong>Range:</strong> {player['pick_range']}
            </div>
        </div>
        """
        cards_html.append(card_html)
    
    # Create grid layout
    grid_html = f"""
    <div style="display: flex; flex-wrap: wrap; justify-content: flex-start; align-items: stretch;">
        {''.join(cards_html)}
    </div>
    """
    
    return HTML(grid_html)

# Create and display player cards
print("TOP AVAILABLE PLAYERS - Card View")
print("=" * 50)
cards_display = create_player_cards(enhanced_df, limit=12)
display(cards_display)

TOP AVAILABLE PLAYERS - Card View


## 5. Availability Curves (Secondary View)

In [15]:
def create_availability_curves(enhanced_df, selected_players=None, my_picks=None):
    """Create line plot showing availability curves for selected players"""
    
    if selected_players is None:
        # Default to top 8 players
        selected_players = enhanced_df.sort_values('overall_rank').head(8)['player_name'].tolist()
    
    if my_picks is None:
        my_picks = []
    
    # Create figure
    fig = go.Figure()
    
    # Generate draft positions for curves
    positions = list(range(1, 51))
    
    # Add curve for each selected player
    for player_name in selected_players:
        player_data = enhanced_df[enhanced_df['player_name'] == player_name]
        if player_data.empty:
            continue
            
        player_rank = player_data.iloc[0]['overall_rank']
        probabilities = [calculate_availability(player_rank, pos) for pos in positions]
        
        fig.add_trace(go.Scatter(
            x=positions,
            y=probabilities,
            mode='lines',
            name=f"{player_name} (#{player_rank})",
            line=dict(width=3),
            hovertemplate='%{fullData.name}<br>Pick %{x}: %{y:.1%}<extra></extra>'
        ))
    
    # Add vertical lines for "my picks"
    for i, pick in enumerate(my_picks[:6]):  # Show first 6 picks
        fig.add_vline(
            x=pick,
            line_dash="dash",
            line_color="red",
            line_width=2,
            annotation_text=f"My Pick {i+1}",
            annotation_position="top"
        )
    
    # Update layout
    fig.update_layout(
        title="Player Availability Curves - Probability vs Draft Position",
        xaxis_title="Draft Position",
        yaxis_title="Probability Available",
        yaxis=dict(tickformat='.0%'),
        hovermode='x unified',
        width=900,
        height=500,
        legend=dict(
            orientation="v",
            yanchor="top",
            y=1,
            xanchor="left",
            x=1.02
        )
    )
    
    return fig

# Create availability curves for top players
top_players = enhanced_df.sort_values('overall_rank').head(8)['player_name'].tolist()
curves_fig = create_availability_curves(enhanced_df, selected_players=top_players, my_picks=my_draft_picks)
curves_fig.show()

## 6. Interactive Dashboard Controls

In [16]:
class DraftDashboard:
    def __init__(self, df):
        self.df = df
        self.drafted_players = set()
        self.my_picks = [8, 17, 32, 41, 56, 65, 80, 89]  # Default 8-team league, pick 8
        self.enhanced_df = None
        self.update_data()
    
    def update_data(self):
        """Update enhanced data based on current state"""
        self.enhanced_df = calculate_player_metrics(self.df, self.my_picks, self.drafted_players)
    
    def set_my_picks(self, picks):
        """Set my draft pick positions"""
        self.my_picks = picks
        self.update_data()
        print(f"Updated my picks: {picks}")
    
    def mark_drafted(self, player_name):
        """Mark a player as drafted"""
        if player_name in self.df['player_name'].values:
            self.drafted_players.add(player_name)
            self.update_data()
            print(f"Marked {player_name} as drafted")
        else:
            print(f"Player {player_name} not found")
    
    def unmark_drafted(self, player_name):
        """Unmark a player as drafted"""
        if player_name in self.drafted_players:
            self.drafted_players.remove(player_name)
            self.update_data()
            print(f"Unmarked {player_name} as drafted")
        else:
            print(f"Player {player_name} was not marked as drafted")
    
    def show_summary_table(self, position_filter='ALL', limit=15):
        """Display compact summary table"""
        table = create_summary_table(self.enhanced_df, position_filter, limit)
        print(f"\nTOP AVAILABLE PLAYERS - {position_filter}")
        print("=" * 60)
        display(table)
    
    def show_player_cards(self, position_filter='ALL', limit=12):
        """Display player cards"""
        cards = create_player_cards(self.enhanced_df, position_filter, limit)
        print(f"\nTOP AVAILABLE PLAYERS - {position_filter} Cards")
        print("=" * 50)
        display(cards)
    
    def show_availability_curves(self, selected_players=None):
        """Display availability curves"""
        if selected_players is None:
            selected_players = self.enhanced_df.sort_values('overall_rank').head(8)['player_name'].tolist()
        
        fig = create_availability_curves(self.enhanced_df, selected_players, self.my_picks)
        fig.show()
    
    def show_dashboard(self, view='cards'):
        """Show complete dashboard"""
        print(f"📊 FANTASY FOOTBALL DRAFT DASHBOARD")
        print(f"Drafted: {len(self.drafted_players)} players | Available: {len(self.enhanced_df)} players")
        print(f"My picks: {self.my_picks[:4]}..." if len(self.my_picks) > 4 else f"My picks: {self.my_picks}")
        print("=" * 70)
        
        if view == 'cards':
            self.show_player_cards()
        elif view == 'table':
            self.show_summary_table()
        elif view == 'curves':
            self.show_availability_curves()
    
    def get_player_list(self):
        """Get list of all available players"""
        return sorted(self.enhanced_df['player_name'].tolist())

# Initialize dashboard
dashboard = DraftDashboard(df)
print("🏈 Draft Dashboard initialized!")
print(f"Default league: 8-team, pick 8")
print(f"My draft picks: {dashboard.my_picks}")

# Show default dashboard
dashboard.show_dashboard(view='cards')

🏈 Draft Dashboard initialized!
Default league: 8-team, pick 8
My draft picks: [8, 17, 32, 41, 56, 65, 80, 89]
📊 FANTASY FOOTBALL DRAFT DASHBOARD
Drafted: 0 players | Available: 300 players
My picks: [8, 17, 32, 41]...

TOP AVAILABLE PLAYERS - ALL Cards


## 7. Interactive Widgets

In [17]:
# Create interactive widgets for the dashboard
player_dropdown = widgets.Dropdown(
    options=dashboard.get_player_list(),
    description='Player:',
    style={'description_width': 'initial'},
    layout={'width': '300px'}
)

position_filter = widgets.Dropdown(
    options=['ALL', 'QB', 'RB', 'WR', 'TE', 'K', 'D/ST'],
    value='ALL',
    description='Position:',
    style={'description_width': 'initial'},
    layout={'width': '150px'}
)

view_selector = widgets.Dropdown(
    options=[('Player Cards', 'cards'), ('Summary Table', 'table'), ('Availability Curves', 'curves')],
    value='cards',
    description='View:',
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)

# Buttons
draft_button = widgets.Button(
    description='Mark as Drafted',
    button_style='danger',
    layout={'width': '140px'}
)

undraft_button = widgets.Button(
    description='Unmark Drafted',
    button_style='warning',
    layout={'width': '140px'}
)

refresh_button = widgets.Button(
    description='Refresh Dashboard',
    button_style='success',
    layout={'width': '140px'}
)

# Output widget
output = widgets.Output()

# Event handlers
def on_draft_clicked(b):
    with output:
        clear_output(wait=True)
        if player_dropdown.value:
            dashboard.mark_drafted(player_dropdown.value)
            # Update dropdown options
            player_dropdown.options = dashboard.get_player_list()

def on_undraft_clicked(b):
    with output:
        clear_output(wait=True)
        if player_dropdown.value:
            dashboard.unmark_drafted(player_dropdown.value)
            # Update dropdown options
            player_dropdown.options = dashboard.get_player_list()

def on_refresh_clicked(b):
    with output:
        clear_output(wait=True)
        dashboard.show_dashboard(view=view_selector.value)

def on_filter_change(change):
    with output:
        clear_output(wait=True)
        if view_selector.value == 'cards':
            dashboard.show_player_cards(position_filter=position_filter.value)
        elif view_selector.value == 'table':
            dashboard.show_summary_table(position_filter=position_filter.value)
        else:
            dashboard.show_availability_curves()

# Wire up event handlers
draft_button.on_click(on_draft_clicked)
undraft_button.on_click(on_undraft_clicked)
refresh_button.on_click(on_refresh_clicked)
position_filter.observe(on_filter_change, names='value')
view_selector.observe(on_filter_change, names='value')

# Create control panel
controls = widgets.VBox([
    widgets.HTML("<h3>🏈 Fantasy Football Draft Dashboard Controls</h3>"),
    widgets.HBox([position_filter, view_selector]),
    widgets.HTML("<h4>Draft Management</h4>"),
    widgets.HBox([player_dropdown]),
    widgets.HBox([draft_button, undraft_button, refresh_button]),
    widgets.HTML("<hr>"),
    output
])

display(controls)

## 8. Quick Usage Examples

### Key Features Implemented:

✅ **80% Reduction in Visual Complexity** - From 15,000 matrix cells to ~150 focused data points  
✅ **Instant Decision Support** - "Who should I draft now?" answered at a glance  
✅ **Mobile-Friendly** - Cards and tables work on small screens  
✅ **Familiar Patterns** - Similar to fantasy sports apps  

### Quick Start Guide:

1. **Set Your Draft Position**: 
   ```python
   dashboard.set_my_picks([8, 17, 32, 41, 56, 65, 80, 89])  # 8-team league, pick 8
   ```

2. **View Top Available Players**:
   ```python
   dashboard.show_dashboard(view='cards')  # Player cards (default)
   dashboard.show_dashboard(view='table')  # Compact table
   ```

3. **Mark Players as Drafted**:
   ```python
   dashboard.mark_drafted("Ja'Marr Chase")
   dashboard.mark_drafted("Bijan Robinson")
   ```

4. **Filter by Position**:
   ```python
   dashboard.show_player_cards(position_filter='RB')
   dashboard.show_summary_table(position_filter='WR')
   ```

5. **View Availability Trends**:
   ```python
   dashboard.show_availability_curves()  # Shows curves for top 8 players
   ```

### Color Coding:
- 🟢 **Green (70%+)**: Very likely available at your pick
- 🟡 **Yellow (30-70%)**: Moderate risk - consider drafting now
- 🔴 **Red (<30%)**: High risk - likely gone before your pick

### Dashboard Views:
- **Player Cards**: Mobile-friendly cards with key metrics
- **Summary Table**: Sortable, color-coded table
- **Availability Curves**: Line charts showing probability trends

Use the interactive controls above to customize your view and track your draft!